In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.sparse import csr_matrix
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import sys
import os
import warnings
import pandas as pd
import json
import string
import re
import nltk
import random
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from scipy.signal import savgol_filter
!pip install kneed
from kneed import KneeLocator
warnings.filterwarnings("ignore")
from kneed import DataGenerator, KneeLocator

In [47]:
# df = pd.read_csv('00.diplomacy_corpus_sep18.csv', encoding='cp1252')
df = pd.read_csv('D:/Dropbox/python_workspace/2021/ism-youtube-gcp-entity/labeled.csv', encoding='utf-8')
print(df.shape)
#df = df[df['message'].notna()]

# df['perception'] = df.apply(lambda row: 0 if row['reciever_perception'] == 'Truth' else 1, axis=1)
# train = df[(df['train_test'] == 'Train') | (df['train_test'] == 'Validation')]
# test = df[df['train_test'] == 'Test']
train, test = train_test_split(df, test_size=0.2)

(211, 17)


## Bag of words and test only logreg

In [48]:
nlp = English()

def is_number(tok):
    try:
        float(tok)
        return True
    except ValueError:
        return False

def spacy_tokenizer(text):
    return [tok.text if not is_number(tok.text) else '_NUM_' for tok in nlp(text)]


In [49]:
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, stop_words=STOP_WORDS, strip_accents='unicode')
corpus = list(train['text'].str.lower())
X_train = vectorizer.fit_transform(corpus)

vectorizer2 = CountVectorizer(tokenizer=spacy_tokenizer, vocabulary=vectorizer.vocabulary_, stop_words=STOP_WORDS, strip_accents='unicode')
X_test = vectorizer2.fit_transform(list(test['text'].str.lower()))

In [50]:
X_train = csr_matrix(X_train)
y_train = train['IsCollab']
X_test = csr_matrix(X_test)
y_test = test['IsCollab']

In [51]:
logreg = LogisticRegression(class_weight = 'balanced', max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [52]:
print(classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.828     0.774     0.800        31
           1      0.500     0.583     0.538        12

    accuracy                          0.721        43
   macro avg      0.664     0.679     0.669        43
weighted avg      0.736     0.721     0.727        43



## Extract tfidf features and test a lot of classifiers

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf_features(df):
    """
        X_train, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test set and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')

    tfidfs = tfidf_vectorizer.fit_transform(df['text'])
    df_tfidf = pd.DataFrame(tfidfs.toarray(), columns=tfidf_vectorizer.get_feature_names())
    return df_tfidf

#df1 = 
df_tfidf = tfidf_features(df)
#print(tfidf_reversed_vocab)
#for i in range(df_tfidf.shape[1]):
#  df['tfidf_{}'.format(i)] = df_tfidf[:, i].toarray()[0]


In [87]:
import csv
print(df_tfidf.shape)
#tf_idf_matrix = df_tfidf.todense()
#tf_idf_df = pd.DataFrame(tf_idf_matrix)
 
#tf_idf_df.columns = tfidf_reversed_vocab
data_top = df_tfidf.head() 
    
# display 
print(data_top)
df_withtfidf = pd.concat([df, df_tfidf], axis=1)

data_top = df_withtfidf.head() 
    
# display 
print(data_top)

(211, 1480)
          &  & cape  & film.  & my  & the  'ask  'ask ben'  'ben  'ben brown'  \
0  0.000000     0.0      0.0   0.0    0.0   0.0        0.0   0.0          0.0   
1  0.091467     0.0      0.0   0.0    0.0   0.0        0.0   0.0          0.0   
2  0.000000     0.0      0.0   0.0    0.0   0.0        0.0   0.0          0.0   
3  0.000000     0.0      0.0   0.0    0.0   0.0        0.0   0.0          0.0   
4  0.000000     0.0      0.0   0.0    0.0   0.0        0.0   0.0          0.0   

   'feel  'feel good    (funky  (funky jazz       (in  (in landscape)  (part  \
0    0.0         0.0  0.000000     0.000000  0.000000        0.000000    0.0   
1    0.0         0.0  0.074952     0.074952  0.078254        0.080138    0.0   
2    0.0         0.0  0.000000     0.000000  0.000000        0.000000    0.0   
3    0.0         0.0  0.075225     0.075225  0.078540        0.080430    0.0   
4    0.0         0.0  0.000000     0.000000  0.000000        0.000000    0.0   

   (part 2)    +    

       VideoID                               Title UploaderUsername  \
0  6X1smU5q1mo       Sephora and Ulta Makeup Haul!       Arden Rose   
1  Cp7_1kJw-JQ                ARRIVING IN PARADISE      FunForLouis   
2  CK963DVi6pI  CALLING MY MUM!! - KSI Animated #4              KSI   
3  jiodLpPKTSc                       PROFESSOR DOG      FunForLouis   
4  InLV5EY-um8     A Brief History Of Dodie Yellow    Ryan O'Connor   

           DateUploaded                        VideoLink  Status  \
0  2013-03-04T01:31:31Z  youtube.com/watch?v=6X1smU5q1mo  public   
1  2016-05-19T20:30:00Z  youtube.com/watch?v=Cp7_1kJw-JQ  public   
2  2015-02-07T20:09:02Z  youtube.com/watch?v=CK963DVi6pI  public   
3  2014-06-17T17:00:06Z  youtube.com/watch?v=jiodLpPKTSc  public   
4  2017-04-20T18:38:47Z  youtube.com/watch?v=InLV5EY-um8  public   

                                         Description  \
0  Hope you guys enjoyed!\n\nFollow me on:\n\nPol...   
1  DAY 1117 // 16TH MAY 2016 //  Dallas United St...

## All preds

In [88]:
main_df = df_withtfidf
feature_start, num_ftrs = 17,1496
print(df_withtfidf.shape)

(211, 2977)


In [89]:
all_classifiers = {}
all_scalers = {}
pd.set_option("display.max_rows", None, "display.max_columns", None)
for feature in ['IsCollab','IsGaming']:
  print('='*80)
  print('For {}:'.format(feature))
  print('Before: {}'.format(len(main_df)), end=' ')
  df = main_df[main_df[feature].notna()]
  print('After: {}'.format(len(df)))
  print('\nClass distribution:')
  print(df[feature].value_counts())
  print()
  vc = dict(df[feature].value_counts())
  minority_class = min(vc, key=vc.get)
  X = df.iloc[:, feature_start:feature_start+num_ftrs]
  y = df[feature]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  classifiers = {'logreg': LogisticRegression(class_weight='balanced'), 
               'knn': KNeighborsClassifier(), 
               'gaussianNB': GaussianNB(),
               'bernoulliNB': BernoulliNB(),
               'adaboost': AdaBoostClassifier(), 
               'grad-boost': GradientBoostingClassifier(),
               'dec-tree': DecisionTreeClassifier(), 
               'linear-svc': LinearSVC(class_weight='balanced'), 
               'c-svc': SVC(class_weight='balanced'), 
               'lda': LinearDiscriminantAnalysis()}
  report = []
  all_classifiers[feature] = classifiers
  all_scalers[feature] = scaler

  

  for method, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report.append([method, 
                 metrics.accuracy_score(y_test, y_pred), 
                 metrics.f1_score(y_test, y_pred),
                 metrics.precision_score(y_test, y_pred),
                 metrics.recall_score(y_test, y_pred), 
                 metrics.f1_score(y_test, y_pred, average='macro'),
                 classification_report(y_test, y_pred, output_dict=True)[str(minority_class)]['f1-score']])
    
    # print('-'*60)
    # print('{:^60}'.format(method))
    # print('-'*60)
    # print(classification_report(y_test, y_pred, digits=3))
    # print()
    
  report = pd.DataFrame(report, columns = ['method', 'accuracy', 'f1', 'precision', 'recall', 'macro-f1', 'minority-f1'])
  print(report)
  print('='*80, '\n\n')

For IsCollab:
Before: 211 After: 211

Class distribution:
0    138
1     73
Name: IsCollab, dtype: int64

        method  accuracy        f1  precision    recall  macro-f1  minority-f1
0       logreg  0.767442  0.642857   0.600000  0.692308  0.735222     0.642857
1          knn  0.790698  0.470588   1.000000  0.307692  0.670077     0.470588
2   gaussianNB  0.720930  0.538462   0.538462  0.538462  0.669231     0.538462
3  bernoulliNB  0.744186  0.352941   0.750000  0.230769  0.596760     0.352941
4     adaboost  0.697674  0.480000   0.500000  0.461538  0.633443     0.480000
5   grad-boost  0.744186  0.521739   0.600000  0.461538  0.673568     0.521739
6     dec-tree  0.720930  0.538462   0.538462  0.538462  0.669231     0.538462
7   linear-svc  0.697674  0.551724   0.500000  0.615385  0.661827     0.551724
8        c-svc  0.767442  0.500000   0.714286  0.384615  0.674242     0.500000
9          lda  0.720930  0.571429   0.533333  0.615385  0.682266     0.571429


For IsGaming:
Before: 2

In [90]:

best_classifiers = {'IsCollab': 'knn', 
                    'IsGaming': 'knn'}

## Predicting for new data: generate tfidf features again

In [95]:
#predicting for new data
#1. generate tfidf features again for your unlabeled dataset
unlabeled_df = pd.read_csv('D:/Dropbox/python_workspace/2021/ism-youtube-gcp-entity/unlabeled.csv', encoding='UTF-8')
print(unlabeled_df.shape)
unlabeled_df_tfidf = tfidf_features(unlabeled_df)
import csv
print(unlabeled_df_tfidf.shape)
#tf_idf_matrix = df_tfidf.todense()
#tf_idf_df = pd.DataFrame(tf_idf_matrix)
 
#tf_idf_df.columns = tfidf_reversed_vocab

## keep only those tfidf features we saw last time
intersect = list(set(df_tfidf.columns).intersection(unlabeled_df_tfidf.columns))
unlabeled_df_tfidf = unlabeled_df_tfidf[intersect]
unlabeled_df_withtfidf = pd.concat([unlabeled_df, unlabeled_df_tfidf], axis=1)

# X = np.random.rand(15783, num_ftrs)
##some features are missing
df_newdata = unlabeled_df_withtfidf.reindex(labels=df_withtfidf.columns,axis=1)
df_newdata =df_newdata.fillna(0)
print(df_newdata.shape)
print(df_newdata.columns)

(20589, 13)
(20589, 46758)
(20589, 2977)
Index(['VideoID', 'Title', 'UploaderUsername', 'DateUploaded', 'VideoLink',
       'Status', 'Description', 'ThumbnailLink', 'UploaderId', 'ViewCount',
       ...
       '✩', '✩ blog/website', '✩ facebook', '✩ instagram', '✩ snapchat!',
       '✩ spotify', '✩ tumblr', '✩ twitter', 'music:',
       'music: https://itunes.apple.com/profile/emmablackery'],
      dtype='object', length=2977)


## Predicting for new data: run trained classifiers on new data

In [100]:

feature_start, num_ftrs = 16,1496
X = df_newdata.iloc[:, feature_start:feature_start+num_ftrs]
print(df_newdata.shape)
for feature in ['IsCollab','IsGaming']:
    X_scaled = all_scalers[feature].transform(X)
    y = all_classifiers[feature][best_classifiers[feature]].predict(X_scaled)
    unlabeled_df['predicted_' + feature] = y
#print the file somewhere

(20589, 2977)
